In [11]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib
import matplotlib.pyplot as plt  # Matlab-style plotting
import seaborn as sns

from sklearn.pipeline import Pipeline
# from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing

from sklearn.preprocessing import OneHotEncoder, LabelEncoder, Normalizer
from sklearn.impute import SimpleImputer

from sklearn.linear_model import LinearRegression, LassoCV, Lasso, Ridge

from sklearn.model_selection import  cross_val_score, StratifiedKFold, learning_curve, train_test_split, KFold
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor


import os
for dirname, _, filenames in os.walk('./'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

./notebook.ipynb
./output.csv
./tcd ml 2019-20 income prediction submission file example (random solutions).csv
./tcd ml 2019-20 income prediction submission file.csv
./tcd ml 2019-20 income prediction test (without labels).csv
./tcd ml 2019-20 income prediction training (with labels).csv
./.ipynb_checkpoints\kernfrom sklearn.ensemble import Randoel5caae9cb4b-checkpoint.ipynb
./.ipynb_checkpoints\notebook-checkpoint.ipynb
./.ipynb_checkpoints\Untitled-checkpoint.ipynb


In [12]:
all_set1= pd.read_csv('./tcd ml 2019-20 income prediction training (with labels).csv') 
no_labels1 = pd.read_csv('./tcd ml 2019-20 income prediction test (without labels).csv')
# no_labels.drop(['Income'], axis=1, inplace = True)
no_labels = no_labels1.drop(['Instance'], axis=1)
all_set = all_set1.drop(['Instance'], axis=1)
all_set = all_set.fillna(np.nan)
no_labels = no_labels.fillna(np.nan)
# print(all_set)

In [13]:
# Get rid of NaN in yor
d=all_set['Year of Record'].values.reshape(-1, 1)
imputer = SimpleImputer(strategy = 'mean')
all_set['Year of Record']= imputer.fit_transform(d)

d=no_labels['Year of Record'].values.reshape(-1, 1)
imputer = SimpleImputer(strategy = 'mean')
no_labels['Year of Record']= imputer.fit_transform(d)

d=all_set['Age'].values.reshape(-1, 1)
imputer = SimpleImputer(strategy = 'mean')
all_set['Age']= imputer.fit_transform(d)

d=no_labels['Age'].values.reshape(-1, 1)
imputer = SimpleImputer(strategy = 'mean')
no_labels['Age']= imputer.fit_transform(d)

######## Profession:
def target_profession(df, mean):
    df['Profession'] = df['Profession'].map(mean)
    return df['Profession']

def target_country(df, mean):
    df['Country'] = df['Country'].map(mean)
    return df['Country']

median = all_set.groupby('Country')['Income in EUR'].median()
medianP = all_set.groupby('Profession')['Income in EUR'].median()
meanCountry = all_set.groupby('Country')['Income in EUR'].mean()
meanProfession = all_set.groupby('Profession')['Income in EUR'].mean()
median1 = no_labels.groupby('Country')['Income'].median()
median2 = no_labels.groupby('Profession')['Income'].median()
# NmeanCountryIncome = no_labels.groupby('Country')['Income'].mean()

all_set['Country'] = target_country(all_set, meanCountry)
all_set['Country'] = all_set['Country'].fillna(value=median)
# mean1?
no_labels['Country'] = target_country(no_labels, meanCountry)
no_labels['Country'] = no_labels['Country'].fillna(value=median1)


all_set['Profession'] = target_profession(all_set, meanProfession)
all_set['Profession'] = all_set['Profession'].fillna(value=medianP)
# mean1?
no_labels['Profession'] = target_profession(no_labels, meanProfession)
no_labels['Profession'] = no_labels['Profession'].fillna(value=median2  )

#########
# Using one-hot encoding on categorial columns

# Adjusting Hair Color
all_set['Hair Color'] = all_set['Hair Color'].fillna('HairUnknown')
all_set['Hair Color'] = all_set['Hair Color'].replace(['0', 'Unknown', 'other'], 'HairUnknown')
all_set = all_set.join(pd.get_dummies(all_set['Hair Color']))
all_set = all_set.drop(['Hair Color'], axis=1)
no_labels['Hair Color'] = no_labels['Hair Color'].fillna('HairUnknown')
no_labels['Hair Color'] = no_labels['Hair Color'].replace(['0', 'Unknown', 'other'], 'HairUnknown')
no_labels = no_labels.join(pd.get_dummies(no_labels['Hair Color']))
no_labels = no_labels.drop(['Hair Color'], axis=1)

# Adjusting the 3 Genders
all_set['Gender'] = all_set['Gender'].fillna('OtherGender')
all_set['Gender'] = all_set['Gender'].replace(['0','unknown', 'other'], 'OtherGender')
all_set = all_set.join(pd.get_dummies(all_set['Gender']))
all_set = all_set.drop(['Gender'], axis=1)
no_labels['Gender'] = no_labels['Gender'].fillna('OtherGender')
no_labels['Gender'] = no_labels['Gender'].replace(['0','unknown', 'other'], 'OtherGender')
no_labels = no_labels.join(pd.get_dummies(no_labels['Gender']))
no_labels = no_labels.drop(['Gender'], axis=1)

# all_set['Gender'] = all_set['Gender'].map({'male':0, 'other':0.5, 'female':1})
all_set['University Degree'] = all_set['University Degree'].fillna('NoDegree')
all_set['University Degree'] = all_set['University Degree'].replace(['0', 'No'], 'NoDegree')
# all_set['University Degree'] = all_set['University Degree'].map({'NoDegree':0, 'Bachelor':0.25, 'Master':0.75, 'PhD':1})
all_set = all_set.join(pd.get_dummies(all_set['University Degree']))
all_set = all_set.drop(['University Degree'], axis=1)
no_labels['University Degree'] = no_labels['University Degree'].fillna('NoDegree')
no_labels['University Degree'] = no_labels['University Degree'].replace(['0', 'No'], 'NoDegree')
# no_labels['University Degree'] = no_labels['University Degree'].map({'NoDegree':0, 'Bachelor':0.25, 'Master':0.75, 'PhD':1})
no_labels = no_labels.join(pd.get_dummies(no_labels['University Degree']))
no_labels = no_labels.drop(['University Degree'], axis=1)

temp = all_set['Body Height [cm]']
all_set = all_set.drop(['Body Height [cm]'], axis=1)
all_set['Height'] = temp

# cant have [] om xgboost
temp = no_labels['Body Height [cm]']
no_labels = no_labels.drop(['Body Height [cm]'], axis=1)
no_labels['Height'] = temp
# spare code
# no_labels['Gender'] = no_labels['Gender'].map({'male':0, 'other':0.5, 'female':1})
# no_labels['University Degree'] = no_labels['University Degree'].map({'Yes':1, 'No':0})
# all_set.isnull().sum()

In [14]:
# all_set = all_set.drop(all_set[(all_set['Income in EUR']>2000000) & (all_set['Year of Record']<1995)].index)
# all_set = all_set.drop(all_set[(all_set['Income in EUR']>2000000) & (all_set['Age']<33)].index)
# all_set = all_set.drop(all_set[(all_set['Income in EUR']>2000000) & (all_set['Age']>80)].index)
# all_set = all_set.drop(all_set[(all_set['Income in EUR']>2700000) & (all_set['Age']>60)].index)

# fig, ax = plt.subplots()
# ax.scatter(x = all_set['Wears Glasses'], y = all_set['Income in EUR'])
# plt.ylabel('Income in EUR', fontsize=13)
# plt.xlabel('Wears Glasses', fontsize=13)
# plt.show()

In [15]:
# number_feature = ['Income in EUR', 'Age', 'Body Height [cm]', 'Wears Glasses', 'Year of Record', 'male','female', 'OtherGender', 'Size of City', 'NoDegree', 'Bachelor', 'Master', 'PhD']
# znumber_feature = ['Age', 'Body Height [cm]', 'Wears Glasses', 'Year of Record', 'male','female', 'OtherGender', 'Size of City', 'NoDegree', 'Bachelor', 'Master', 'PhD']
drop_features = ['Black', 'Blond', 'Brown', 'HairUnknown', 'Red', 'NoDegree', 'OtherGender', 'Wears Glasses']
X = all_set.drop(drop_features, axis=1)
Z = no_labels.drop(drop_features, axis=1)
# X = all_set
# Z = no_labels
Z.drop('Income', axis=1, inplace = True)
# X = X.fillna(X.mean())
# Z = Z.fillna(X.mean())
# Z = Z.fillna(method='ffill')
# Normalising

# # Size of city
# d=X['Size of City'].values.reshape(-1, 1)
# min_max_scaler = preprocessing.MinMaxScaler()
# X['Size of City']= min_max_scaler.fit_transform(d)

# # Size of city
# d=Z['Size of City'].values.reshape(-1, 1)
# min_max_scaler = preprocessing.MinMaxScaler()
# Z['Size of City']= min_max_scaler.fit_transform(d)

# # age
# d=X['Age'].values.reshape(-1, 1)
# min_max_scaler = preprocessing.MinMaxScaler()
# X['Age']= min_max_scaler.fit_transform(d)

# # age
# d=Z['Age'].values.reshape(-1, 1)
# min_max_scaler = preprocessing.MinMaxScaler()
# Z['Age']= min_max_scaler.fit_transform(d)

# # yor
# d=X['Year of Record'].values.reshape(-1, 1)
# min_max_scaler = preprocessing.MinMaxScaler()
# X['Year of Record']= min_max_scaler.fit_transform(d)

# # yor
# d=Z['Year of Record'].values.reshape(-1, 1)
# min_max_scaler = preprocessing.MinMaxScaler()
# Z['Year of Record']= min_max_scaler.fit_transform(d)

# # Height
# d=X['Height'].values.reshape(-1, 1)
# min_max_scaler = preprocessing.MinMaxScaler()
# X['Height']= min_max_scaler.fit_transform(d)

# # Height
# d=Z['Height'].values.reshape(-1, 1)
# min_max_scaler = preprocessing.MinMaxScaler()
# Z['Height']= min_max_scaler.fit_transform(d)

X = X.fillna(X.mean())
Z = Z.fillna(X.mean())

In [16]:
#  For submissions
X_train = X.drop(['Income in EUR'], axis=1)
y_train = X['Income in EUR']

# Split 70/30 - for tests
sorted_train, sorted_test = train_test_split(X, test_size=0.3,random_state=0)

# X_train = sorted_train.drop(['Income in EUR'], axis=1)
# y_train = sorted_train['Income in EUR']
# X_test = sorted_test.drop(['Income in EUR'], axis=1)
# y_test = sorted_test['Income in EUR']

In [17]:
# #Using Pearson Correlation
# plt.figure(figsize=(12,10))
# cor = sorted_train.corr()
# sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
# plt.show()
# #Correlation with output variable
# cor_target = abs(cor["Income in EUR"])
# print(cor_target)

In [18]:
# reg = LassoCV()
# reg.fit(X_train, y_train)
# print("Best alpha using built-in LassoCV: %f" % reg.alpha_)
# print("Best score using built-in LassoCV: %f" %reg.score(X_train,y_train))
# coef = pd.Series(reg.coef_, index = X_train.columns)
# print("Lasso picked " + str(sum(coef != 0)) + " variables and eliminated the other " +  str(sum(coef == 0)) + " variables")
# imp_coef = coef.sort_values()
# import matplotlib
# matplotlib.rcParams['figure.figsize'] = (8.0, 10.0)
# imp_coef.plot(kind = "barh")
# plt.title("Feature importance using Lasso Model")

In [ ]:
# import re
# import xgboost as xgb
# from xgboost.sklearn import XGBRegressor

# linreg = RandomForestRegressor(n_estimators=10)
# # fit the model to the training data (learn the coefficients)

reg_algo =  AdaBoostRegressor(base_estimator=RandomForestRegressor(), n_estimators=100)

# reg_algo = LinearRegression()

# reg_algo = xgb.XGBRegressor(objective ='reg:squarederror', n_estimators = 10)


reg_algo.fit(X_train, y_train)

# y_pred = reg_algo.predict(X_test)
# print(y_pred)
# np.sqrt(mean_squared_error(y_test, y_pred)) #61582??? 75970

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: F

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
D:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: F

In [ ]:
z_predict = reg_algo.predict(Z)
print(z_predict)
# [ 68056.84430952  22308.38636086 122793.98068889 ... 140460.61764565
#  785403.71618078 473919.2276137 ]
submission_df = pd.DataFrame()
submission_df['Instance'] = no_labels1['Instance']
submission_df['Income'] = z_predict.tolist()
print(submission_df)
# # submission_df.tail()
submission_df.to_csv('output.csv',index=False)
print("done")